In [1]:
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Rect, Text, HoverTool
import pandas as pd

In [2]:
# output_notebook()

In [4]:
def ticker_func(num):
    if num == 0:
        return "19:00"
    elif (int(str(num)[-1]) == 4) or (int(str(num)[-1]) == 9):
        return "11:00"
    elif (int(str(num)[-1]) == 3) or (int(str(num)[-1]) == 8):
        return "13:00"
    elif (int(str(num)[-1]) == 2) or (int(str(num)[-1]) == 7):
        return "15:00"
    elif (int(str(num)[-1]) == 1) or (int(str(num)[-1]) == 6):
        return "17:00"
    else:
        return "09:00"
    
def color_pick(i):
    if i % 2 == 0:
        return "lavenderblush"
    return "pink"

schedule_df = pd.read_csv("../output_data/schedule_student_numbers.csv").set_index("Time")

rooms_df = pd.read_csv("../input_data/rooms.csv").rename(columns={"Zaalnummber":"room", "Max. capaciteit" : "capacity"})
rooms_df["capacity"] = rooms_df["capacity"].apply(str)
room_dict = rooms_df.to_dict('index')

output_file("output_data/schedule.html")

schedule = Plot(width=2000, height=2000)

# determine rectangle dimensions
w = np.full(35, .95)
h = np.full(35, .8)

# keep track of row in schedule dataframe
df_row_index = 0

# fill the schedule top to bottom (Monday to Friday)
for day_index in range(0, 25, 5)[::-1]: # starten met maandag; 20 = Monday, 15, 10, 5, 0 = Friday
    x_values = np.array([])
    y_values = np.array([])
    courses = []
    
    # loop over the time slots on 1 day
    for y in range(day_index, 5 + day_index)[::-1]: # starten met 09:00; 4 = 09:00, 3, 2, 1, 0 = 17:00    
        
        # loop over the rooms
        for x in range(7):
        
            x_values = np.append(x_values, x)
            y_values = np.append(y_values, y + .5)
            
            # remove all excess course information
            course_name = str(schedule_df.iloc[df_row_index][x].split("|")[0])
            
            # append the course name
            courses.append(course_name if len(course_name) > 1 else "")
           
        # move one row down in the schedule df
        df_row_index += 1

    # add 1 day's rectangles to schedule
    rect_source = ColumnDataSource(dict(x=x_values, y=y_values, w=w, h=h))
    rectangles = Rect(x="x", y="y", width="w", height="h", fill_color=color_pick(day_index))
    schedule.add_glyph(rect_source, rectangles)

    
    # add text
    # make sure the text aligns nicely with rectangles
    text_x_values = x_values - .45
    text_y_values = y_values + .1
    
    text_source = ColumnDataSource(dict(x=text_x_values, y=text_y_values, text=courses))
    my_text = Text(x="x", y="y", text="text")
    my_text.text_font_size = {'value': '11px'}
    schedule.add_glyph(text_source, my_text)
    
# add x and y axis - ???
xaxis = LinearAxis()
schedule.add_layout(xaxis, 'above')
yaxis = LinearAxis(axis_label='Time slots')
schedule.add_layout(yaxis, 'left')

# ???
schedule.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
schedule.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

# create time tick labels
schedule.yaxis.ticker = np.arange(0, 25 + 1)
schedule.yaxis.major_label_overrides = {num : ticker_func(num) for num in range(25 + 1)} # zowel 0 als 19:00 moeten een tick krijgen, 1 meer dus dan het aantal tijdslots

# create room tick labels
schedule.xaxis.major_label_overrides = {num : (room_dict[num]["room"] + " | " + room_dict[num]["capacity"]) for num in range(7)}
schedule.xaxis.minor_tick_line_color = None

# TODO: de lesson waarden achterhalen -> df maken met type, group, students en malus points?
# lekker hoveren 
hover = HoverTool()
hover.tooltips = """
    <div>
        <div><strong>Type: </strong>Lab</div>
        <div><strong>Group: </strong>3</div>
        <div><strong>Students: </strong>18</div>
        <div><strong>Malus points: </strong>454</div>
    </div>
"""

schedule.add_tools(hover)

# show the results
show(schedule)

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/schedule.html'

### TO DO
- daadwerkelijke code importeren zodat ik zelf veel dingen niet nog een keer doe
- hovering (ook nog welke studenten?)
- 2e as voor dagen (lukt niet naar mn zin. Kan de getallen rechts niet aanpassen zonder dat ze links ook veranderen)
- efficientere / betere code met numpy en pandas, minder for-loops
- (selection tool voor individuele roosters)